In [ ]:
import io, os, sys, ssl, time, json, socket, struct, random, datetime, threading
import serial, pigpio, picamera
from adafruit_motorkit import MotorKit
from IPython.display import clear_output

elevation_pin = 21
rotation_pin  = 20  
gpio_port     = 8889
image_port    = 54321
sensor_port   = 54324
control_port  = 54322  
server_ip = 'yzlab3.chem.nyu.edu'
server_cert_self_signed  = 'secret/server.crt' 
client_certs_self_signed = 'secret/rover.crt'
client_key_self_signed   = 'secret/rover.key'
retry_in = 5 # seconds
# SSL context
context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH, cafile=server_cert_self_signed)
context.load_cert_chain(certfile=client_certs_self_signed, keyfile=client_key_self_signed)

In [ ]:
def startGPIOdaemon():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)                                            #2 Second Timeout
    result = sock.connect_ex(('127.0.0.1', gpio_port))
    if result == 0:
        print( 'pigpiod is running' )
    else:
        !sudo pkill pigpiod
        !sudo pigpiod -p 8889 ### Start PiGPIO Server / Deamon    http://abyz.me.uk/rpi/pigpio/pigpiod.html
        print( 'pigpiod was just started' ) 
    sock.close() 

In [ ]:
def image_link_daemon():
# https://picamera.readthedocs.io/en/release-1.13/recipes1.html#capturing-to-a-network-stream
# https://picamera.readthedocs.io/en/release-1.13/recipes2.html#web-streaming
    #camera = picamera.PiCamera(resolution=(640, 480))  
    camera = picamera.PiCamera(resolution=(320, 240)) 
    camera.rotation = 270
    camera.framerate = 25
    camera.start_preview() 
    time.sleep(3)  
    while True:
        while True:
            try:
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip) 
                ss.settimeout(5)  
                print('[Image Link] Trying to connect the server')
                ss.connect((server_ip, image_port)) 
                print('[Image Link] Established a connection') 
                img_io = ss.makefile('wb')
                stream = io.BytesIO()
                break
            except (ConnectionRefusedError, ConnectionResetError):
                print(f'[Image Link] Failed to connect the server. Retry in {retry_in} seconds')
                time.sleep(retry_in)
                continue
        try:
            for foo in camera.capture_continuous(stream, 'jpeg', use_video_port=False):  # video_port = low quality
                #camera.annotate_text = f'Celsius:{sensor.Temperature.rstrip()} {dt}'  
                img_io.write(struct.pack('<L', stream.tell())) # size of frame
                img_io.flush() # send 
                stream.seek(0)
                img_io.write(stream.read()) 
                img_io.flush()
                stream.seek(0)
                stream.truncate()  
        except socket.timeout as e: 
            ss.shutdown(socket.SHUT_RDWR)
            ss.close()
            print(f'[Image Link] Time out and closed. Retry in {retry_in} seconds:', e)
            time.sleep(retry_in) 
        except (ConnectionResetError, BrokenPipeError) as e:
            print(f'[Image Link] Server ended the connection. Retry in {retry_in} seconds:', e)
            time.sleep(retry_in)

In [ ]:
def sensor_link_daemon():
    time.sleep(6)
    Compass = serial.Serial(port='/dev/ttyUSB0', baudrate=115200, timeout=3) # serial commu with Arduino Uno
    GPS     = serial.Serial(port='/dev/ttyS0'  , baudrate=9600  , timeout=3) # serial commu with GPS chip 
    if Compass.readline(): # Discard the first reading since it could be corrupted 
        print('Compass is ready') 
    if GPS.readline(): # Discard the first reading since it could be corrupted  
        print('GPS is ready') 
    while True: 
        # try until a sensor link is established
        while True: 
            try: 
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip) 
                ss.settimeout(5)  
                print('[Sensor Link] Trying to connect the server')
                ss.connect((server_ip, sensor_port)) 
                print('[Sensor Link] Succeed a connection') 
                break
            except (ConnectionRefusedError, ConnectionResetError):
                print(f'[Sensor Link] Failed to connect the server and retry the connection in {retry_in} seconds')
                time.sleep(retry_in)
        try:
            while True:
                data = b''
                while True:
                    _ = GPS.readline()
                    if _[:6] == b'$GPRMC':  # http://aprs.gids.nl/nmea/
                        data = _ 
                    elif data and _[:6] == b'$GPGGA': 
                        data += _ 
                        data += Compass.readline()
                        break
                ss.send(struct.pack('<H', len(data))) 
                ss.send(data) 
                time.sleep(2) 
        except socket.timeout as e: 
            ss.shutdown(socket.SHUT_RDWR)
            ss.close()
            print(f'[Sensor Link] Time out and closed. Reestablish in {retry_in} seconds:', e)
            time.sleep(retry_in) 
        except (ConnectionResetError, BrokenPipeError) as e:
            print(f'[Sensor Link] Server ended the connection. Reestablish in {retry_in} seconds:', e)
            time.sleep(retry_in)

In [ ]:
def control_link_daemon(kit):
    time.sleep(9)
    l = struct.calcsize('<hf')
    pi = pigpio.pi(port=gpio_port) ### Start PiGPIO Client
    while True:
        try: 
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip)
            ss.settimeout(30)
            print('[Control link] Trying to connect the server') 
            ss.connect((server_ip, control_port)) 
            print('[Control link] Succeed a connection')
            while True:
                code, value = struct.unpack('<hf', ss.recv(l))  
                try:
                    if code in [4, 5]: time.sleep(0.05) # give motor some time 
                    if code == -1:
                        pi.set_servo_pulsewidth(rotation_pin, 0) 
                        pi.set_servo_pulsewidth(elevation_pin, 0) 
                    elif code == 2: pi.set_servo_pulsewidth(rotation_pin, value) 
                    elif code == 3: pi.set_servo_pulsewidth(elevation_pin, value)
                    elif code == 4: kit.motor3.throttle = value
                    elif code == 5: kit.motor4.throttle = value
                except OSError as e:
                    print(code, value, e)
        except socket.timeout as e:
            ss.shutdown(socket.SHUT_RDWR) 
            ss.close()
            print(f'[Control link] Failed to receive any data from server so that control channel closes and will reopen in {retry_in} seconds', e)
            time.sleep(retry_in)      
        except struct.error as e:
            ss.shutdown(socket.SHUT_RDWR) 
            ss.close()
            print(f'[Control link] Control Data is ill-formed', e)
            time.sleep(retry_in)
        except ConnectionRefusedError:
            print(f'[Control link] Failed to connect the server and retry in {retry_in} seconds')
            time.sleep(retry_in)

In [ ]:
startGPIOdaemon()
#image_link_thread   = threading.Thread(target=image_link_daemon); image_link_thread.start(); print('image_link_daemon', image_link_thread.getName()) 
#sensor_link_thread  = threading.Thread(target=sensor_link_daemon); sensor_link_thread.start(); print('sensor_link_daemon', sensor_link_thread.getName())
control_link_thread = threading.Thread(target=control_link_daemon, args=(MotorKit(),)); control_link_thread.start(); print('control_link_daemon', control_link_thread.getName())